<p style="text-align: center;font-size: 40pt">
Overview of lidars
<img src="images/car_with_lidar.jpg" width="75%">
</p>

# Introduction

Lidars are one of the most common sensors one can find on autonomous vehicles. The reason behind their popularity is that they are becoming more and more affordable and provide rich 3D information about the geometry of the surroundings. In this lesson, you will learn the basics of how lidars work and what are their limitations. 

# Basic principle

- bouncing light pulses
- many lasers (or channels or beams) + rotating mirror
- known angles + known distance (with back-and-forth time and speed of light) -> spherical coordinates -> can be converted to cartesian coordinates
- point coordinates are in lidar frame

# Now, in real life

- angle of incidence / object reflectance -> absorption / diffusion / reflection -> returned pulse intensity is decreased or null
- laser beam is a cone, not a line -> precision decreases with distance
- multiple returns -> multiple points with one pulse
- sensor is assumed to be static during scans -> it isn't

# Error sources

- multiple returns averaged -> shadow points
- specular reflections -> virtual points
- angle uncertainty (mechanism) -> error in angle
- peak detection + clock accuracy -> error in time
- sensor temperature -> error in time
- bias (distance + angle of incidence) -> error in time
- refraction -> error in time

# Lidar specifications

- laser beam divergence -> (degrees)
- range/angle precision (influenced by errors and distance + beam divergence) -> (meters)
- range/angle accuracy (influenced by errors) -> (meters)

- horizontal field of view -> the horizontal area captured by the sensor (degrees)
- horizontal (or azimuth) angular resolution -> smallest horizontal angle between two successive points (degrees)
- horizontal points -> the number of horizontal points generated per scan
- horizontal points -> horizontal field of view / horizontal angular resolution

- vertical field of view -> the vertical area captured by the sensor (degrees)
- vertical angular resolution -> smallest vertical angle between two successive points (degrees)
- vertical points -> the number of rows of points generated per scan, also called channels or beams
- vertical points = vertical field of view / vertical angular resolution

- scan rate -> number of scans generated per second
- points per second -> total number of points generated in a second
- points per second = horizontal points * vertical points * scan rate

- laser class safety

- minimum range -> smallest distance from target from which the sensor can operate (meters)
- maximum range -> biggest distance from target from which the sensor can operate (meters)

interaction between metrics, but marketing people tend to hide it